# Building Firmware

In [ ]:
PLATFORM = "CWLITEARM"
CRYPTO_TARGET="NONE"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/your_project
make PLATFORM=$1 CRYPTO_TARGET=$2

# Capture

## Setup

In [ ]:
%run "Helper Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "Helper Scripts/Setup_Target_Generic.ipynb"

In [ ]:
# uncomment based on your target
fw_path = "../../hardware/victims/firmware/path/to/binary"
#%run "Helper Scripts/Program_XMEGA.ipynb"
%run "Helper Scripts/Program_STM.ipynb"
#%run "Helper Scripts/No_Programmer.ipynb"

In [ ]:
program_target(scope, fw_path)

## Create Project

In [ ]:
project = cw.createProject("projects/Your project", overwrite=True)
tc = project.getTraceFormat()

## Basic Capture Loop (For Analyzer)

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time

ktp = AcqKeyTextPattern_Basic(target=target)

N = 50  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here

    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    
    # note you may want:
    # num_char = target.ser.inWaiting()
    # response = target.ser.read(num_char)
    _ = target.readOutput()  # clears the response from the serial port
    #traces.append(scope.getLastTrace())
    tc.addTrace(scope.getLastTrace(), text, "", key)

## Saving Project Traces

In [ ]:
from datetime import datetime
import copy

starttime = datetime.now()
prefix = starttime.strftime('%Y.%m.%d-%H.%M.%S') + "_"
tc.config.setConfigFilename(project.datadirectory + "traces/config_" + prefix + ".cfg")
tc.config.setAttr("prefix", prefix)
tc.config.setAttr("date", starttime.strftime('%Y-%m-%d %H:%M:%S'))
tc.closeAll()
project.save()

## Disconnect

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

# Analysis (With Analyzer)

## Loading Project

In [ ]:
import chipwhisperer as cw
project = cw.openProject("projects/Your project")

## Setting up Attack

In [ ]:
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit, SBox_output

attack = CPA()

leak_model = AES128_8bit(SBox_output)
attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(project.traceManager())
attack.setTraceStart(0)
attack.setTracesPerAttack(project.traceManager().numTraces())
attack.setIterations(1)
attack.setReportingInterval(10)
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
attack.setPointRange((0, -1))

## Attack With Table

In [ ]:
import pandas as pd
key = project.traceManager().getKnownKey(0)
def format_stat(stat):
    return str("{:02X}<br>{:.3f}".format(stat[0], stat[2]))

def color_corr_key(row):
    global key
    ret = [""] * 16
    for i,bnum in enumerate(row):
        if bnum[0] == key[i]:
            ret[i] = "color: red"
        else:
            ret[i] = ""
    return ret

from IPython.display import clear_output
import numpy as np
        
def stats_callback():
    attack_results = attack.getStatistics()
    attack_results.setKnownkey(key)
    stat_data = attack_results.findMaximums()
    df = pd.DataFrame(stat_data).transpose()
    clear_output(wait=True)
    display(df.head().style.format(format_stat).apply(color_corr_key,axis=1))
    
attack_results = attack.processTracesNoGUI(stats_callback)

## Plot Data

In [ ]:
plot_data = cw.analyzerPlots(attack_results)

In [ ]:
rets = []
for i in range(16):
    rets.append(plot_data.outputVsTime(i))

In [ ]:
for bnum in range(16):
    ret = plot_data.pgeVsTrace(bnum)

In [ ]:
for bnum in range(16):
    ret = plot_data.corrVsTrace(bnum)

# Plotting With Bokeh

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()
p.line(xdata, ydata, line_color="red")
show(p)